In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gp import *
import time

In [2]:
z_train = np.array([0,1,5]).reshape(-1,1)
y_train = np.array([7,2,3]).reshape(-1,1)
z_query = np.arange(-10,10,0.1).reshape(-1,1)

theta0 = [1,1,1] # Kernel variables

model = GPR(z_train, y_train, covariance_function=KernelFunction, theta=theta0)
model_prior = GPR(None, None, covariance_function=KernelFunction, theta=theta0)
mean_prior, stb_prior = model_prior.predict(z_query, std=True)
# Before ML optimization
mean_b, std_b = model.predict(z_query, std=True)

# ML optimization
model.maximize_likelyhood()

# After optimization
mean_a, std_a = model.predict(z_query, std=True)


Size of feature training data = (3, 1)
Size of output training data = (3, 1)


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
a = None
a == None

a = np.array([1,2])
a == None

In [ ]:
plt.figure(figsize=(10,6), dpi=100)
plt.subplot(1,3,1)
plt.plot(z_query, mean_prior)
plt.fill_between(z_query.reshape(-1), mean_prior.reshape(-1) - stb_prior, mean_prior.reshape(-1) + stb_prior,
                 color='gray', alpha=0.2)
plt.xlabel('Velocity in x [ms-1]')
plt.ylabel('Acceleration error in x [ms-2]')
plt.title(f'GP Before ML optimization')
plt.legend(('GP interpolation', 'Measured samples', 'Training samples'))

plt.subplot(1,3,2)
plt.plot(z_query, mean_b)

plt.plot(z_train, y_train, 'x')
plt.fill_between(z_query.reshape(-1), mean_b.reshape(-1) - std_b, mean_b.reshape(-1) + std_b,
                 color='gray', alpha=0.2)
plt.xlabel('Velocity in x [ms-1]')
plt.ylabel('Acceleration error in x [ms-2]')
plt.title(f'GP Before ML optimization')
plt.legend(('GP interpolation', 'Measured samples', 'Training samples'))


plt.subplot(1,3,3)
plt.plot(z_query, mean_a)

plt.plot(z_train, y_train, 'x')
plt.fill_between(z_query.reshape(-1), mean_a.reshape(-1) - std_a, mean_a.reshape(-1) + std_a,
                 color='gray', alpha=0.2)
plt.xlabel('Velocity in x [ms-1]')
plt.ylabel('Acceleration error in x [ms-2]')
plt.title(f'GP Before ML optimization')
plt.legend(('GP interpolation', 'Measured samples', 'Training samples'))

In [ ]:
def one_dim_GPR(dim):
    
    z = d_loader.get_z(training=False)
    y = d_loader.get_y(training=False)


    z_train = d_loader.get_z(training=True)
    y_train = d_loader.get_y(training=True)


    if dim == 'x':
        # x velocity
        z_train = z_train[:,7].reshape(-1,1)
        z = z[:,7].reshape(-1,1)

        y_train = y_train[:,0].reshape(-1,1)
        y = y[:,0].reshape(-1,1)
        
    if dim == 'y':
        # y velocity
        z_train = z_train[:,8].reshape(-1,1)
        z = z[:,8].reshape(-1,1)

        y_train = y_train[:,1].reshape(-1,1)
        y = y[:,1].reshape(-1,1)
        
    if dim == 'z':
        # z velocity
        z_train = z_train[:,9].reshape(-1,1)
        z = z[:,9].reshape(-1,1)

        y_train = y_train[:,2].reshape(-1,1)
        y = y[:,2].reshape(-1,1)
    
    theta0 = [6,1,1] # Kernel variables
    x_query = np.arange(-20,20,0.05).reshape(-1,1)

    model = GPR(z_train, y_train, covariance_function=RBF, theta=theta0)
    # Before ML optimization
    mean_b, std_b = model.predict(x_query, std=True)

    # Calculate the RMS over all samples
    mean_test_before = model.predict(z, std=False)
    rms_before = np.sqrt(np.mean((y - mean_test_before)**2))

    # ML optimization
    model.maximize_likelyhood()

    # After optimization
    mean_a, std_a = model.predict(x_query, std=True)

    # Calculate the RMS over all samples
    mean_test_after = model.predict(z, std=False)
    rms_after = np.sqrt(np.mean((y - mean_test_after)**2))

    print(f'RMS before = {rms_before}')
    print(f'RMS after = {rms_after}')
    
    plt.figure(figsize=(10,6), dpi=100)
    plt.subplot(1,2,1)
    plt.plot(x_query, mean_b)
    plt.scatter(z, y, s=1, c='g')
    plt.plot(z_train, y_train, 'x')
    plt.fill_between(x_query.reshape(-1), mean_b.reshape(-1) - std_b, mean_b.reshape(-1) + std_b,
                     color='gray', alpha=0.2)
    plt.xlabel('Velocity in x [ms-1]')
    plt.ylabel('Acceleration error in x [ms-2]')
    plt.title(f'GP Before ML optimization, rms = {rms_before:.2f}')
    plt.legend(('GP interpolation', 'Measured samples', 'Training samples'))


    plt.subplot(1,2,2)
    plt.plot(x_query, mean_a)
    plt.scatter(z, y, s=1, c='g')
    plt.plot(z_train, y_train, 'x')
    plt.fill_between(x_query.reshape(-1), mean_a.reshape(-1) - std_a, mean_a.reshape(-1) + std_a,
                     color='gray', alpha=0.2)
    plt.xlabel('Velocity in x [ms-1]')
    plt.ylabel('Acceleration error in x [ms-2]')
    plt.title(f'GP Before ML optimization, rms = {rms_after:.2f}')
    plt.legend(('GP interpolation', 'Measured samples', 'Training samples'))

In [ ]:
one_dim_GPR('x')

In [ ]:
one_dim_GPR('y')

In [ ]:
one_dim_GPR('z')